# T-725 Natural Language Processing: Lab 9
In today's lab, we will be working with **semantic analysis**.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* Select `"Runtime" > "Run all"` to run the code in this notebook.

In [32]:
import nltk
nltk.download('book_grammars')

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Package book_grammars is already up-to-date!


True

## Feature-based grammars

Consider the following context free grammar:

```
S   ->   NP VP
NP  ->   Det N
VP  ->   V

Det  ->  'this' | 'these'
N    ->  'dog' | 'dogs'
V    ->  'run' | 'runs'
```

While it can parse and generate the sentences "*this dog runs*" and "*these dogs run*", it can also generate ungrammatical sentences such as "*this dogs runs*" and "*these dog run*". We can edit the grammar to ensure that verbs and their subjects, as well as nouns and determiners, agree in number:

```
S -> NP_SG VP_SG | NP_PL VP_PL
NP_SG -> Det_SG N_SG
NP_PL -> Det_PL N_PL
VP_SG -> V_SG
VP_PL -> V_PL

Det_SG -> 'this'
Det_PL -> 'these'
N_SG -> 'dog'
N_PL -> 'dogs'
V_SG -> 'runs'
V_PL -> 'run'
```

Unfortunately, this comes at the cost of doubling the number of productions. The grammar will quickly grow in size as we continue to add to it. Let's look at a more concise way to define the revised grammar:

```
S -> NP[NUM=?n] VP[NUM=?n]
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
VP[NUM=?n] -> V[NUM=?n]

Det[NUM=sg] -> 'this'
Det[NUM=pl] -> 'these'
N[NUM=sg] -> 'dog'
N[NUM=pl] -> 'dogs'
V[NUM=sg] -> 'runs'
V[NUM=pl] -> 'run'
```

This is known as a *feature-based grammar* (see [Chapter 9](http://www.nltk.org/book/ch09.html) of the NLTK book). Instead of creating specific categories for the number, we create a *feature* called `NUM` which we can assign to different categories. We also allow *variables* over feature values, which are used to state constraints. In the grammar above, `NUM=?n` specifies that the number can be either singular, `sg`, or plural, `pl`. The production `S -> NP[NUM=?n] VP[NUM=?n]` means that `NUM` must take the same value for `NP` and `VP`.

## First-order logic
If we know that all dogs bark, and that Cyril is a dog, then we also know that Cyril barks. This line of reasoning can be formally stated using [first-order logic](http://www.nltk.org/book/ch10.html#sec-fol):

* $\forall{x}(dog(x) \implies barks(x))$: All dogs bark.
  * The $\forall$ symbol means "for all" and $A\implies B$ means "if $A$ is true, then $B$ is also true", or "$A$ implies $B$".
* $dog(Cyril)$: Cyril is a dog
* $dog(Cyril) \implies barks(Cyril)$: If Cyril is a dog, then Cyril barks.
* $barks(Cyril)$: Cyril barks.

In the NLTK, these statements are expressed in the following way:
* `all x.(dog(x) -> barks(x))`
* `dog(Cyril)`
* `dog(Cyril) -> barks(Cyril)`
* `barks(Cyril)`

You can perform syntax-driven semantic analysis of natural language input sentences by building an NLTK parser from a feature-based grammar that contains semantic attachments as features. Such a parser constructs statements like the ones above as it parses sentences.

Building the parser is as simple as calling `nltk.load_parser("<fcfg filename>")` and assigning the resulting parser object to a variable like `parser`. NLTK creates the right kind of parser by inspecting the grammar. Let's take a look at one such grammar, `simple-sem.fcfg`:

In [33]:
!cat /root/nltk_data/grammars/book_grammars/simple-sem.fcfg

## Natural Language Toolkit: sem3.fcfg
##
## Alternative simple grammar with transitive verbs and 
## quantifiers for the book. 
## 
## Author: Ewan Klein <ewan@inf.ed.ac.uk> 
## URL: <http://nltk.sourceforge.net>
## For license information, see LICENSE.TXT


% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=s

Let's parse a few sentences using this grammar:

In [34]:
parser = nltk.load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)

sentences = ['all dogs bark', 'Angus gives a bone to every dog']

for sentence in sentences:
  print(sentence)
  tokens = sentence.split()

  for tree in parser.parse(tokens):
    #print(tree)
    print(tree.label()['SEM'])
  print()

all dogs bark
all x.(dog(x) -> bark(x))

Angus gives a bone to every dog
all z15.(dog(z15) -> exists z14.(bone(z14) & give(angus,z14,z15)))



# Assignment
Answer the following questions and hand in your solution in Canvas before 8:30 Monday morning, October 30th. Remember to save your file before uploading it.

## Question 1
Add to the lexicon in `my_grammar` below so that additional sentences of your choosing can be parsed (e.g., *some people play an instrument*). You do not need to modify or add any grammar rules, you only need to add new **lexical rules** at the end of the string that is assigned to `my_grammar`. Confirm that you get the FOL representation you expect.

**Note**: If you have the proper lexical rules and still cannot parse your sentence, you may need to rephrase it. For example, this simple grammar cannot parse the sentence "*some people play instruments*", but it can parse "*some people play an instrument*" or "*some people play some instruments*".

In [35]:
my_grammar = r"""
% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'

Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'

N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.boy(x)>] -> 'boy'
N[NUM=sg,SEM=<\x.bone(x)>] -> 'bone'
N[NUM=sg,SEM=<\x.ankle(x)>] -> 'ankle'
N[NUM=sg,SEM=<\x.dog(x)>] -> 'dog'
N[NUM=pl,SEM=<\x.dog(x)>] -> 'dogs'

IV[NUM=sg,SEM=<\x.bark(x)>,TNS=pres] -> 'barks'
IV[NUM=pl,SEM=<\x.bark(x)>,TNS=pres] -> 'bark'
IV[NUM=sg,SEM=<\x.walk(x)>,TNS=pres] -> 'walks'
IV[NUM=pl,SEM=<\x.walk(x)>,TNS=pres] -> 'walk'
TV[NUM=sg,SEM=<\X x.X(\y.chase(x,y))>,TNS=pres] -> 'chases'
TV[NUM=pl,SEM=<\X x.X(\y.chase(x,y))>,TNS=pres] -> 'chase'
TV[NUM=sg,SEM=<\X x.X(\y.see(x,y))>,TNS=pres] -> 'sees'
TV[NUM=pl,SEM=<\X x.X(\y.see(x,y))>,TNS=pres] -> 'see'
TV[NUM=sg,SEM=<\X x.X(\y.bite(x,y))>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\X x.X(\y.bite(x,y))>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'give'

P[+to] -> 'to'

# New Lexical Rules
#############################
N[NUM=pl,SEM=<\x.person(x)>] -> 'people'
TV[NUM=pl,SEM=<\X x.X(\y.play(x,y))>,TNS=pres] -> 'play'
N[NUM=sg,SEM=<\x.instrument(x)>] -> 'instrument'
N[NUM=sg,SEM=<\x.instrument(x)>] -> 'instruments'
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(my_grammar)
parser = nltk.parse.FeatureEarleyChartParser(grammar)

# Parse your sentences here, after modifying the grammar above.
# Hint: Copy the code from the last parsing example above, you can use the same code to assign sentences and printing out the parse results.
sentences = ['some people play an instrument', 'some people play some instruments']

for sentence in sentences:
  print(sentence)
  tokens = sentence.split()

  for tree in parser.parse(tokens):
    #print(tree)
    print(tree.label()['SEM'])
  print()

some people play an instrument
exists x.(person(x) & exists z16.(instrument(z16) & play(x,z16)))

some people play some instruments
exists x.(person(x) & exists z17.(instrument(z17) & play(x,z17)))



## Question 2
Our parser generates FOL representations for sentences in English, but it is not capable of evaluating them. You can build a model in the NLTK using the exact same set-based model-theoretic semantics as the text book explains in Chapter 15.2 and shows in Figure 15.2. Note that an NLTK model already specifies a certain denotation and interpretation. That is, you include specific Objects, Properties and Relations and associate them with certain elements of the domain, sets and sets of tuples, respectively.

**Building a model**

You build the model object from a special string representation of the model using the `nltk.Valuation.fromstring()` function, which returns a so-called valuation. If you store the valuation in a variable, you can pass it, along with its domain, into the constructor of a new Model: `m = nltk.Model(val.domain, val)`.


In [36]:
model_representation = """
spot => s
dog => {s}
bark => {s}
"""

# Build the model
val = nltk.Valuation.fromstring(model_representation)
var_assignments = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)

**Evaluating FOL statements**

You can then start evaluating FOL statements (in string format) against the model like this: `m.evaluate("all x.(dog(x) -> bark(x))", nltk.Assignment(val.domain))` (The second parameter contains any variable substitutions if there are any, in this case there are none and we are passing the default assignments).

In [37]:
# Evaluate a semantic representation
m.evaluate("all x.(dog(x) -> bark(x))", var_assignments)

True

More examples can be found in [Chapter 10](http://www.nltk.org/book/ch10.html#truth-in-model) of the NLTK book.

Add to the model representation below so that it can be used to evaluate the sentences you had in mind in Question 1. Test your model by evaluating various statements.

In [38]:
model_representation = """
angus => a
cyril => c
irene => i
boy => {a}
girl => {i}
dog => {c}
walk => {i, c}
see => {(a, i), (c, a), (i, c)}
person => {a, i}
instrument => {c}
play => {(a, c), (i, c)}
"""

# Build your model and evaluate your statements here, after modifying the model above.

val = nltk.Valuation.fromstring(model_representation)
var_assignments = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
m.evaluate("exists x.(person(x) & exists z12.(instrument(z12) & play(x,z12)))", var_assignments)

True

## Question 3
With the parser from Question 1 and model from Question 2, we can directly evaluate the truth value of English sentences, by:

1. Translating an English sentence into a FOL statement like we did in Question 1.
2. Using the resulting expression as the first argument to the `evaluate()` method of the model. However, we first need to convert the expression into a string (`semantic_expression = str(semantic_expression)`).
3. Evaluating the truth value of the statement with the `evaluate()` method of the model, like we did in Question 2.

Evaluate the truth value of several English sentences that are supported by the grammar from Question 1 and model from Question 2.

In [59]:
# parser from question 1
def parse_sentence(sentence):
  tokens = sentence.split()

  for tree in parser.parse(tokens):
    #print(tree)
    return str(tree.label()['SEM'])
# model from question 2
def get_model(sentence):
  val = nltk.Valuation.fromstring(model_representation)
  var_assignments = nltk.Assignment(val.domain)
  m = nltk.Model(val.domain, val)
  return m, var_assignments

def evaluate(sentence):
  parsed = parse_sentence(sentence)
  (model, var_assignments) = get_model(parsed)
  return model.evaluate(parsed, var_assignments)

sentences = [
    'all people walk',
    'Angus walks',
    'some dogs walk',
    'some people walk',
]

for sentence in sentences:
    print(f'{sentence} - {evaluate(sentence)}')

all people walk - False
Angus walks - False
some dogs walk - True
some people walk - True
